# Sur les 18 pages info procédure de licenciement combien ont trouvé leur réponse

**Définition** : Ce que je veux faire

**Formule** : 

## On commence par le chargement des données
on décommente ce dont on a besoin

In [ ]:
import pandas as pd
import time
import json
import math
from tqdm.notebook import tqdm
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from src.elasticsearch_connector import ElasticsearchConnector
#from typing import List, Dict


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

## Si on veut lire en requêtant Elastic Search

In [ ]:
es_connector = ElasticsearchConnector(env='monolog')

QUERY = {
  "query": {
    "bool": { 
      "must": [
        {
          "range": {
            "logfile": {
              "gte": "2023-02-01",
              "lt": "2023-03-01"
            }
          }
        }
      ]
    }
  }
}

In [ ]:
logs = es_connector.execute_query(QUERY, "logs-new")

In [ ]:
logs = logs.append(logs2)

In [ ]:
logs.shape

In [ ]:
logs.head()

In [ ]:
def filter_df_startwith_url(df: pd.DataFrame, url):
    return df[df['url'].str.match(f'^{url}', na=False)].copy()

In [ ]:
def format_url_and_get_month_and_datetime_columns(df: pd.DataFrame):
    df['url'] = df['url'].str.split('#').str[0].str.split('?').str[0]
    df['month'] = pd.DatetimeIndex(df['logfile']).month
    df['datetime'] = df['timestamp'].apply(datetime.fromtimestamp)
    return df

In [ ]:
def clean_dataset_from_useless_rows(df: pd.DataFrame):
    #df = df.drop_duplicates(subset=['idVisit', 'url', 'type']).copy()
    return df[df['type'].isin(['visit_content', 'outil', 'themes', 'select_result', 'search', 'home',
                                    'pagecc_clickcontrib', 'modeles_courriers'])]

In [ ]:
logs_formated = format_url_and_get_month_and_datetime_columns(logs)

In [ ]:
#logs_contrib = filter_df_startwith_url(logs_formated, 'https://code.travail.gouv.fr/contribution/')
#logs_april_contrib_without_duplicates = logs_contrib.drop_duplicates(subset=['idVisit', 'url', 'type']).copy()

In [ ]:
logs_formated.shape[0]

# Début de l'explo

Commentaire

In [ ]:
logs_information_licenciement = logs_formated[
    (logs_formated['url'].notna()) & 
    (logs_formated['url'].str.contains('information/')) &
    (logs_formated['url'].str.contains('licenciement'))
].copy()

In [ ]:
logs_information_licenciement

In [ ]:
logs_information_licenciement[logs_information_licenciement['feedbackType']=='Fermeture de l’établi pour non conformité 27/12/22\nTout le personnel doit être licencié \nChômage partiel refusé le 17 février \nLa demande du 24/1']

In [ ]:
logs_information_licenciement[logs_information_licenciement['idVisit']=='32720971']

In [ ]:
logs_information_licenciement[
    logs_information_licenciement['feedbackType'].notna()
].feedbackType.value_counts().to_frame()